In [ ]:
import os
import sys

INPUTFOLDER="../PAKs/cycle2/raw"
OUTPUTFOLDER="../PAKs/cycle2/extracted"

try:
    os.mkdir(OUTPUTFOLDER)
except FileExistsError:
    pass


menu=f"../program-template.nabu"
list="../program-list.csv"
with open(menu, "rb") as f:
    menu = bytearray(f.read())
    with open(list, "rt") as list:
        for x in range(0,0x300):
        #for line in list:    
            #file, name = line.split(",")
            #file = file.strip()
            #name = name.strip()
            file = "{0:06X}".format(x)
            name = "{0:06X}".format(x)

            filename = f"{INPUTFOLDER}/{file}.nabu"
            outFilename = f"{OUTPUTFOLDER}/{name}.nabu"
            print(F"{filename} -> {outFilename}")
            newProgram = bytearray(menu)
            writeStart = 0xE1
            readStart = 0x08
            
            try:
                with open(filename, "rb") as inFile:
                    length = os.path.getsize(filename)-readStart
                    programFile = inFile.read()
                    print(f"{filename} {name}")    
                    for i in range(0, length):
                        #print(f"Write: {writeStart+i} Read: {readStart+i}")
                        newProgram[writeStart+i] = programFile[readStart+i]
                with open(outFilename, "wb") as outFile:
                    outFile.write(newProgram)
            except:
                pass




    

    